## Ejemplo: Datos EEUU

Las operaciones de ``merge`` y ``join`` surgen con mayor frecuencia cuando se combinan datos de diferentes fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre EEUU y sus poblaciones, en el que nuestro principal objetivo será obtener un ranking con la densidad de población total de cada estado en 2010.

Los datos pueden ser descargados de http://github.com/jakevdp/data-USstates/:

Comenzamos importando librerías y leyendo los datos:

In [46]:
import pandas as pd
import numpy as np

In [47]:
# Leemos los datos:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [48]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [49]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Dada esta información, digamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EEUU según su densidad de población de 2010.
Obviamente, esto es algo que podemos obtener con los datos que tenemos, pero tendremos que combinar los conjuntos de datos para encontrar el resultado.

Comenzaremos con una combinación de varios-a-uno que nos dará el nombre completo del estado dentro de la población, para lo que necesitaremos realizar la unión en función de la columna "state/region" de "pop" y la columna de "abbreviation" de "abbrevs".

En este caso, como queremos asegurarnos de que no se pierdan datos debido a que no existan cruces, tataremos el cruce como "outer".

In [51]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # eliminamos la columna con la misma info que la otra de cruce: state/regio
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Verificaremos si existen registros que no crucen, que lo podemos hacer buscando filas con nulos:

In [52]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

En este caso, vemos que la información de ``population`` tiene nulos, ¿por qué podría pasar? ¡Investiguémoslo!

Empezamos sacando algunos de estos registros donde "population" es nulo:

In [56]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


Y con esto podremos sacar todos los valores únicos de 'state/region' para los 'population' nulos. Aqui se hace necesario mencionar que, para sacar los valroes únicos de una serie, podemos utilizar el método ``unique()``.

In [25]:
merged.loc[merged['population'].isnull(), 'state/region'].unique()

array(['PR'], dtype=object)

In [26]:
merged.loc[merged['population'].isnull(), 'year'].unique()

array([1990, 1991, 1993, 1992, 1994, 1995, 1996, 1998, 1997, 1999],
      dtype=int64)

Parece que todos los valores nulos de población son de Puerto Rico antes del año 2000; lo cual se deberá, probablemente, a que estos datos no están disponibles en la fuente original.

Más importante aún, vemos también que algunas de las nuevas entradas de ``estado`` también son nulas, lo que significa que no había una entrada correspondiente en la clave ``"abbrevs"``.

¡Averigüemos qué regiones carecen de esta coincidencia!

In [57]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos inferir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (EE. UU.), mientras que estas entradas no aparecen en la columna de cruce ("state") que estamos utilizando de abreviatura.

Podemos solucionarlos rápidamente completando las entradas correspondientes:

In [28]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Como podéis ver, la columna ``state`` ya no tiene nulos :)

Ahora podemos hacer un ``merge`` del resultado con el área de trabajo con un procedimiento similar a lo que hemos visto antes.

Examinando nuestro resultados, lo que tendremos que hacer será un cruce con la columna 'state' de ambos ``DataFrame``:

In [29]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


De nuevo, verificaremos si hay nulos para ver si hubo alguna discrepancia:

In [30]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Pues sí que hay nulos, solo que esta vez están localizados (además de en "population") en la columna ``area``. Podemos echar un vistazo del mismo modo que lo hemos hecho antes para ver qué regiones se ven afectadas por ello:

In [31]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Como podemos comprobar, el valor que le falta a la columna ``area`` es el correspondiente al valor "United States", que es la totalidad del área de EEUU, lo cual se tendría que corresponder con la suma de las áreas de sus territorios.

Por lo tanto, podríamos rellenar este valor haciendo la suma del resto. Sin embargo, en este caso no inresa demasiado, por lo que simplemente nos descartaremos de este registro:

In [32]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Ahora que tenemos todos los datos que necesitamos, podemos responder a la pregunta que nos estábamos haciendo desde el principio: ¿cuál es la densidad de población de cada estado?

Para ello, filtramos para quedarnos con aquellos datos del año 2010 y que hagan referencia a la población total del estado (fíjate que ahora lo que tenemos son registros de varios años y de la población total y menos de 18 años):

In [41]:
data2010 = final[(final["year"] == 2010) & (final["ages"] == 'total')]
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora nos calculamos la densidad de población a partir de los datos que tenemos y la mostramos en orden para obtener nuestro ranking.

Para ello, podemos actuar de varias formas, ya sea añadiendo la ``Serie`` creada al ``DataFrame`` resultado u operando directamente sobre el ``Series`` creado. Para este ejemplo, optaremos por la segunda. Por tanto, cambiaremos el índice a la columna 'state' para dar los resultados basados en este camop:

In [42]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

ORdenamos y damos el TOP:

In [58]:
density.sort_values(ascending=False, inplace=True)
density.head(10)

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
Massachusetts            621.815538
Maryland                 466.445797
Delaware                 460.445752
New York                 356.094135
Florida                  286.597129
dtype: float64

El resultado es una clasificación de los estados de EEUU, más "Washington, DC" (que se administra como un distrito federal) y "Puerto Rico", en orden de su densidad de población de 2010, en residentes por milla cuadrada.

Podemos ver que, con mucho, la región más densa en este conjunto de datos es "Washington, DC" (District of Columbia)., mientras que entre los estados como tal, el más denso es "New Jersey".

Del mismo modo, pese a que lo que nos pedían era el TOP, también podemos consultar el final de la lista:

In [59]:
density.tail(10)

state
Utah            32.677188
Nevada          24.448796
Nebraska        23.654153
Idaho           18.794338
New Mexico      16.982737
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con bastante diferencia, es Alaska, con una densidad de prácticamente 1 residente por milla cuadrada.

Este tipo de combinaciones desordenadas de datos son una tarea común cuando se intenta responder preguntas utilizando fuentes de datos del mundo real.